In [1]:
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

f = open("../../dataset/sense/dict_sense-keys", 'rb')
dict_sense_keys = pickle.load(f)
f.close()

f = open("../../dataset/sense/dict_word-sense", 'rb')
dict_word_sense = pickle.load(f)
f.close()

f = open('../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()
word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

with open('/data/aviraj/dataset/raw_preprocess_train','rb') as f:
    data=pickle.load(f)

with open('/data/aviraj/dataset/fulldata_vocab_sense','rb') as f:
    vocab_lex=pickle.load(f)

lex2id = dict((s, i) for i,s in enumerate(vocab_sense))
id2lex = dict((i, s) for i,s in enumerate(vocab_sense))

print(len(vocab_sense))

num_senses = len(vocab_sense)
max_sent_size = 200

46


In [33]:
_pos = []
for i in range(len(data)):
    for pp in data[i][4]:
        _pos.append(pp)
        
pos_count = Counter(_pos)
pos_count = pos_count.most_common()
vocab_pos = [pp for pp, c in pos_count]
pos2id = dict((s, i) for i,s in enumerate(vocab_pos))

In [2]:
data_y1 = []
data_y2 = []
data_y3 = []
for i in range(len(data)):
    if (len(data[i][1])<=200):
        for j in range(len(data[i][2])):
            if data[i][2][j] is not None:
                data_y1.append(dict_sense_keys[data[i][2][j]][3])
                data_y2.append(dict_sense_keys[data[i][2][j]][4])
                data_y3.append(dict_sense_keys[data[i][2][j]][5])

sense_count1 = Counter(data_y1)
sense_count1 = sense_count1.most_common()
sense_count2 = Counter(data_y2)
sense_count2 = sense_count2.most_common(312)
sense_count3 = Counter(data_y3)
sense_count3 = sense_count3.most_common(1051)

dict_sense_count1 = dict(sense_count1)
dict_sense_count2 = dict(sense_count2)
dict_sense_count3 = dict(sense_count3)

In [3]:
print(len(sense_count1), len(sense_count2), len(sense_count3))

46 312 1051


In [4]:
data_x = []
data_pos = []
data_label1 = []
data_label2 = []
data_label3 = []

for i in range(len(data)):
    if not all(np.array(data[i][2])==None) and (len(data[i][1])<=200):
        data_label1.append([ss if ss is not None and dict_sense_keys[ss][3] in dict_sense_count1 else None for ss in data[i][2]])
        data_label2.append([ss if ss is not None and dict_sense_keys[ss][4] in dict_sense_count2 else None for ss in data[i][2]])
        data_label3.append([ss if ss is not None and dict_sense_keys[ss][5] in dict_sense_count3 else None for ss in data[i][2]])
        data_x.append(data[i][1])
        data_pos.append(data[i][4])

In [5]:
split_label1 = []
split_label2 = []
split_label3 = []
index1 = []
index2 = []
index3 = []

for jj, lab in enumerate(data_label1):
        min_idx = np.argmin([dict_sense_count1[dict_sense_keys[lab[i]][3]] if lab[i] is not None else np.inf for i in range(len(lab))  ]) 
        if(min_idx != np.inf):
            index1.append(jj)
            split_label1.append(lab[min_idx])

for jj, lab in enumerate(data_label2):
        min_idx = np.argmin([dict_sense_count2[dict_sense_keys[lab[i]][4]] if lab[i] is not None else np.inf for i in range(len(lab))  ]) 
        if(min_idx != np.inf):
            index2.append(jj)
            split_label2.append(lab[min_idx])

for jj, lab in enumerate(data_label3):
        min_idx = np.argmin([dict_sense_count3[dict_sense_keys[lab[i]][5]] if lab[i] is not None else np.inf for i in range(len(lab))  ]) 
        if(min_idx != np.inf):
            index3.append(jj)
            split_label3.append(lab[min_idx])

print(len(split_label1))
print(len(split_label2))
print(len(split_label3))

850093
850093
850093


In [5]:
def data_prepare(x, pos, y, sense_count, lex_cond=False, pos_cond=False):
    num_examples = len(x)
    
    vocab_sense = [s for s, c in sense_count]
    sense2id = dict((s[0], i) for i,s in enumerate(sense_count))
    
    xx = np.zeros([num_examples, max_sent_size], dtype=int)
    xx_mask = np.zeros([num_examples, max_sent_size], dtype=bool)
    ss_mask = np.zeros([num_examples, max_sent_size], dtype=bool)
    yy = np.zeros([num_examples,max_sent_size], dtype=int)
    y_lex = np.zeros([num_examples, max_sent_size], dtype=int)
    y_pos = np.zeros([num_examples, max_sent_size], dtype=int)
        
    for j in range(num_examples):
        for i in range(max_sent_size):
            if(i>=len(x[j])):
                break
            w = x[j][i]
            s = y[j][i]
            p = pos[j][i]
            xx[j][i] = word2id[w] if w in word2id else word2id['UNKNOWN_TOKEN']
            xx_mask[j][i] = True
            ss_mask[j][i] = True if s in vocab_sense else False
            yy[j][i] = sense2id[s] if s in vocab_sense else 0
            if(lex_cond):
                y_lex[j][i] = lex2id[dict_sense_keys[s][3]] if in vocab_lex else len(vocab_lex)
            if(pos_cond):
                y_pos[j][i] = pos2id[p] if p in vocab_pos else len(vocab_pos)
        
    return xx, xx_mask, ss_mask, yy, y_lex, y_pos

In [ ]:
def train_val_data(name, index, split_label, data_label, sense_count, lex_cond=False, pos_cond=False):
    
    index_train, index_val, label_train_id, label_val_id = train_test_split(index, split_label, train_size=0.8, shuffle=True, stratify=split_label, random_state=0)

    data_x = np.array(data_x)
    data_label = np.array(data_label)
    
    x_train = data_x[index_train]
    y_train = data_label[index_train]
    x_val = data_x[index_val]
    y_val = data_label[index_val]
    
    pos_train = []
    pos_val = []
    
    if(pos_cond):
        pos_train = data_pos[index_train]
        pos_val = data_pos[index_val]

    x_id_train, mask_train, sense_mask_train, y_id_train, lex_train, pos_id_train  = data_prepare(x_train, pos_train, y_train, sense_count, lex_cond=lex_cond, pos_cond=pos_cond)
    x_id_val, mask_val, sense_mask_val, y_id_val, lex_val, pos_id_val = data_prepare(x_val, pos_val, y_val, sense_count, lex_cond=lex_cond, pos_cond=pos_cond)

    train_data = {'x':x_id_train,'x_mask':mask_train, 'sense_mask':sense_mask_train, 'y':y_id_train, 'lex':lex_train, 'pos':pos_id_train}
    val_data = {'x':x_id_val,'x_mask':mask_val, 'sense_mask':sense_mask_val, 'y':y_id_val, 'lex':lex_val, 'pos':pos_id_val}
    
    with open('/data/aviraj/dataset/train_val_data/all_word_'+ name,'wb') as f:
        pickle.dump([train_data,val_data], f)
    
    print(len(x_id_train)+len(x_id_val))

In [6]:
train_val_data(split_label1)
train_val_data(split_label2)
train_val_data(split_label3)

/users/btech/aviraj/cs771/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [8]:
list_sample = [('1:19', 10000),
 ('1:17', 10000),
 ('2:34', 10000),
 ('2:33', 10000),
 ('1:27', 10000),
 ('2:37', 8000),
 ('1:24', 8000),
 ('1:08', 8000),
 ('1:12', 7000),
 ('1:22', 5000),
 ('2:29', 5000),
 ('1:05', 3000),
 ('1:16', 3000),
 ('1:25', 3000),
 ('1:20', 3000),
 ('1:13', 2000),
 ('2:43', 1100),
 ('3:44', 1000)]

dict_sample = dict(list_sample)

In [14]:
sm = RandomOverSampler(ratio=dict_sample)

index_train = np.array(index_train).reshape(-1, 1)
sampled_index, _ = sm.fit_sample(index_train, label_train_id)

print(len(sampled_index), len(index_train))

741155 680074


In [15]:
count = Counter(_)
count = count.most_common()
count

[('3:00', 59395),
 ('5:00', 56790),
 ('1:04', 52909),
 ('1:10', 40702),
 ('1:09', 38131),
 ('1:14', 33103),
 ('2:32', 30711),
 ('2:31', 28800),
 ('2:41', 25662),
 ('2:42', 25318),
 ('1:07', 25124),
 ('1:18', 21228),
 ('2:30', 19801),
 ('1:26', 19794),
 ('1:06', 19728),
 ('1:28', 19211),
 ('2:40', 15804),
 ('2:38', 12146),
 ('2:36', 12035),
 ('1:21', 11834),
 ('1:15', 11142),
 ('3:01', 10656),
 ('1:19', 10000),
 ('1:17', 10000),
 ('2:34', 10000),
 ('2:33', 10000),
 ('1:27', 10000),
 ('2:35', 9885),
 ('1:11', 9117),
 ('2:39', 8526),
 ('2:37', 8000),
 ('1:08', 8000),
 ('1:24', 8000),
 ('1:03', 7675),
 ('1:23', 7114),
 ('1:12', 7000),
 ('2:29', 5000),
 ('1:22', 5000),
 ('1:05', 3000),
 ('1:20', 3000),
 ('1:16', 3000),
 ('1:25', 3000),
 ('1:13', 2000),
 ('4:02', 1714),
 ('2:43', 1100),
 ('3:44', 1000)]

In [16]:
sampled_index_train = np.array(sampled_index).reshape(1, -1)
sampled_x_train = data_x[sampled_index_train[0]]
sampled_y_train = data_label[sampled_index_train[0]]

print(len(sampled_x_train), len(sampled_y_train))

741155 741155


In [17]:
sampled_x_id_train, sampled_mask_train, sampled_sense_mask_train, sampled_y_train = data_prepare(sampled_x_train, sampled_y_train)

with open('/data/aviraj/dataset/sampled_train_data','wb') as f:
    pickle.dump([sampled_x_id_train, sampled_mask_train, sampled_sense_mask_train, sampled_y_train],f)